## TP
aca hay que poner una introducción supongo idk

In [1]:
import numpy as np
import sys

from alc import *
# TODO esto todo va a estar en alc cuando terminemos y hay que borrarlo
sys.path.append(".")
sys.path.append("../src")
from labo00_auxiliares import *
from labo01_errores_igualdad import *
from labo02_TLs_basicas import *
from labo03_normas import *
from labo04_LU import *
from labo05_QR import *
from labo06_AVs import *
from labo07_markov import *
from labo08_SVD import *


no se explicar que hacemos las pruebas

In [2]:
import os
os.getcwd()[-3:]


'/TP'

In [11]:
Xt, Yt, Xv, Yv = cargarDataset("cats_and_dogs")
print(Xt.shape)
print(Xv.shape)
print(Yt, Yt.shape)
print(Yv, Yv.shape)

(1536, 2000)
(1536, 1000)
[[0 0 0 ... 1 1 1]
 [1 1 1 ... 0 0 0]] (2, 2000)
[[0 0 0 ... 1 1 1]
 [1 1 1 ... 0 0 0]] (2, 1000)
